We have data from an Audiobook app in .csv format. Logically, it relates only to the audio versions of books. each customer in the database has made a purchase at least once, thats why customer is in database. we want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.

The main idea is that if a customer has a low probability of coming back. there is no reason to spend any money on advertising to customer. if we can focus our efforts only on customers that are likely to convert again. we can make great savings. moreover, this model can identify the most important metrics for a customer to come back again. identifying new customer creates value and growth opportunities. 

In [2]:
import numpy as np
from sklearn import preprocessing


In [6]:
raw_data = np.loadtxt(r"C:\Users\nnhp\Downloads\Audiobooks_data (1).csv", delimiter= ",")# download the dataset/file from the data folder, pass the path of that file
all_unscaled_inputs = raw_data[:,1:-1]
all_targets = raw_data[:,-1]

balance the dataset

In [7]:
num_one_targets = int(np.sum(all_targets))
counter_Zero_targets= 0
indices_to_remove = []

for i in range(all_targets.shape[0]):
    if all_targets[i] ==0:
        counter_Zero_targets +=1
        if counter_Zero_targets > num_one_targets:
            indices_to_remove.append(i)
            
priors_unscaled_inputs_equal = np.delete(all_unscaled_inputs, indices_to_remove, axis=0)
priors_targets_equals = np.delete (all_targets, indices_to_remove, axis = 0)


Standardize the inputs

In [9]:
inputs_scaled = preprocessing.scale(priors_unscaled_inputs_equal)

Shuffle the data

In [14]:
indices_shuffled = np.arange(inputs_scaled.shape[0])
np.random.shuffle(indices_shuffled)

inputs_shuffled= inputs_scaled[indices_shuffled]
targets_shuffled= priors_targets_equals[indices_shuffled]

Split the datasets into train, validation, and test 

In [17]:
count_sample = inputs_shuffled.shape[0]
train_count_sample = int(0.8*count_sample)
validation_count_sample = int(0.1*count_sample)
test_count_sample = count_sample - train_count_sample - validation_count_sample

train_inputs = inputs_shuffled[:train_count_sample]
train_targets = targets_shuffled[:train_count_sample]

validation_inputs= inputs_shuffled[train_count_sample:train_count_sample+validation_count_sample]
validation_targets= targets_shuffled[train_count_sample:train_count_sample+validation_count_sample]

test_inputs = inputs_shuffled[train_count_sample+validation_count_sample:]
test_targets = targets_shuffled[train_count_sample+validation_count_sample:]

print(np.sum(train_targets), train_count_sample, np.sum(train_targets)/train_count_sample)
print(np.sum(validation_targets), validation_count_sample, np.sum(validation_targets)/validation_count_sample)
print(np.sum(test_targets), test_count_sample, np.sum(test_targets)/test_count_sample)

1805.0 3579 0.5043308186644314
234.0 447 0.5234899328859061
198.0 448 0.4419642857142857


Save the three datasets in *.npz

In [19]:
np.savez('Audiobooks_train_data', final_inputs = train_inputs, final_targets= train_targets)
np.savez('Audiobooks_validation_data', final_inputs = validation_inputs, final_targets= validation_targets)
np.savez('Audiobooks_test_data', final_inputs = test_inputs, final_targets= test_targets)

Create the machine learning algorithm

In [ ]:
import tensorflow as tf

In [20]:
npz= np.load('Audiobooks_train_data.npz')

train_x1_inputs = npz['final_inputs'].astype(np.float)
train_y1_targets = npz['final_targets'].astype(np.int)

npz= np.load('Audiobooks_validation_data.npz')

validation_x2_inputs= npz['final_inputs'].astype(np.float)
validation_y2_targets= npz['final_targets'].astype(np.int)

npz= np.load('Audiobooks_test_data.npz')

test_x2_inputs= npz['final_inputs'].astype(np.float)
test_y2_targets= npz['final_targets'].astype(np.int)



Model

Outline, Optimizers, loss, early stopping and training

In [110]:
inputs_size = 10
outputs_size = 2
hidden_layers_size= 50

tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layers_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layers_size, activation = 'relu'),
    tf.keras.layers.Dense(outputs_size, activation = 'softmax')
])

model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(),
             optimizer= tf.keras.optimizers.Adam(),
             metrics=['accuracy'])

batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping()

model.fit(train_x1_inputs,
         train_y1_targets,
         batch_size= batch_size,
         epochs= max_epochs,
         callbacks = [early_stopping],
         validation_data= (validation_x2_inputs, validation_y2_targets),
         verbose=2)

Epoch 1/100
36/36 - 10s - loss: 0.6130 - accuracy: 0.6558 - val_loss: 0.5373 - val_accuracy: 0.7069
Epoch 2/100
36/36 - 0s - loss: 0.4699 - accuracy: 0.7622 - val_loss: 0.4599 - val_accuracy: 0.7606
Epoch 3/100
36/36 - 0s - loss: 0.4128 - accuracy: 0.7837 - val_loss: 0.4212 - val_accuracy: 0.7808
Epoch 4/100
36/36 - 0s - loss: 0.3867 - accuracy: 0.7907 - val_loss: 0.4018 - val_accuracy: 0.8210
Epoch 5/100
36/36 - 0s - loss: 0.3737 - accuracy: 0.7958 - val_loss: 0.3870 - val_accuracy: 0.7942
Epoch 6/100
36/36 - 0s - loss: 0.3605 - accuracy: 0.8044 - val_loss: 0.3805 - val_accuracy: 0.7987
Epoch 7/100
36/36 - 0s - loss: 0.3523 - accuracy: 0.8100 - val_loss: 0.3710 - val_accuracy: 0.8031
Epoch 8/100
36/36 - 0s - loss: 0.3484 - accuracy: 0.8092 - val_loss: 0.3644 - val_accuracy: 0.7987
Epoch 9/100
36/36 - 0s - loss: 0.3462 - accuracy: 0.8094 - val_loss: 0.3628 - val_accuracy: 0.8031
Epoch 10/100
36/36 - 0s - loss: 0.3405 - accuracy: 0.8092 - val_loss: 0.3646 - val_accuracy: 0.8031


Test the model

In [108]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 6ms/step - loss: 0.3601 - accuracy: 0.8214


In [109]:
print('\nTest loss:{0:.2f}, Test accuracy:{1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss:0.36, Test accuracy:82.14%
